In [1]:
# ML Libraries
import torch
from torch import nn
import torch.nn.functional as F
import torchaudio
from torchaudio import transforms
from torch.utils.data import Dataset
from torchmetrics.text.wer import WordErrorRate
from torchmetrics.text.cer import CharErrorRate
from torch.cuda.amp import autocast, GradScaler


# Support Libraries
import math
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm
import gc
from itertools import groupby

# Python Scripts
from conf_model import ConformerEncoder, LSTMDecoder
from conf_utils import *

In [2]:
total_devices = torch.cuda.device_count()
print(f"Available GPU Devices : {total_devices}")
DEVICES = [i for i in range(0,total_devices)]
print(f"Using devices : {DEVICES}")
DEVICE = f"cuda:{DEVICES[0]}"

Available GPU Devices : 1
Using devices : [0]


In [3]:
TRAIN_BS = 20*len(DEVICES)
TEST_BS = 20*len(DEVICES)
EPOCHS = 1000
NUM_WORKERS = 4*len(DEVICES)
start_epoch = 0

metadata_path = "/media/rathna/New Volume/datasets/LJSpeech-1.1/metadata.tsv"
metadata = pd.read_csv(metadata_path, sep = '\t', header = None)

metadata = metadata[metadata[2]<=16].reset_index(drop=True)

metadata[0] = "/media/rathna/New Volume/datasets/LJSpeech-1.1/wavs/" + metadata[0]

metadata = metadata.sample(frac=1).reset_index(drop=True)

print(metadata)

In [4]:
metadata_train = pd.read_csv("/media/rathna/New Volume/datasets/Librispeech/metadata_train_clean.tsv", sep = '\t', header = None)
metadata_dev = pd.read_csv("/media/rathna/New Volume/datasets/Librispeech/metadata_test_clean.tsv", sep = '\t', header = None)

metadata_train = metadata_train[metadata_train[2]<=16].reset_index(drop=True)


metadata_train = metadata_train.sample(frac=1).reset_index(drop=True)

print(metadata_train)

metadata_dev = metadata_dev[metadata_dev[2]<=16].reset_index(drop=True)



metadata_dev = metadata_dev.sample(frac=1).reset_index(drop=True)

print(metadata_dev)

                                                        0  \
0       /media/rathna/New Volume/datasets/Librispeech/...   
1       /media/rathna/New Volume/datasets/Librispeech/...   
2       /media/rathna/New Volume/datasets/Librispeech/...   
3       /media/rathna/New Volume/datasets/Librispeech/...   
4       /media/rathna/New Volume/datasets/Librispeech/...   
...                                                   ...   
124491  /media/rathna/New Volume/datasets/Librispeech/...   
124492  /media/rathna/New Volume/datasets/Librispeech/...   
124493  /media/rathna/New Volume/datasets/Librispeech/...   
124494  /media/rathna/New Volume/datasets/Librispeech/...   
124495  /media/rathna/New Volume/datasets/Librispeech/...   

                                                        1       2      3  
0       or my dread of its doing them both harm and i ...  15.765  16000  
1       when about a fortnight old however they genera...  13.260  16000  
2       the real evils indeed of emmas sit

USE THIS TO BUILD THE TOKENIZER AND MARK DOWN BEFORE RUNNING THE CODE

import tokenizers
import os

vocab_size = 1004 #CAN SPECIFY ANY MAXIMUM VALUE
tokenizer_type = "wpe"
dst_folder = "/media/rathna/New Volume/word_piece"
text_path = "/media/rathna/New Volume/word_piece/libri_text_460.txt"
dataset = "libri"
tokenizer_dir = os.path.join(dst_folder, 'tokenizer_{}_{}_v{}').format(dataset, tokenizer_type, vocab_size)

if not os.path.exists(tokenizer_dir):
    os.makedirs(tokenizer_dir)

tokenizer = tokenizers.BertWordPieceTokenizer(lowercase=False) #if true, treats upper and lower case as separate tokens

tokenizer.train(text_path, vocab_size=vocab_size)
tokenizer.save_model(tokenizer_dir)

del tokenizer
gc.collect()

In [5]:
# opening the file in read mode
my_file = open("/media/rathna/New Volume/word_piece/tokenizer_libri_wpe_v1004/vocab.txt", "r")
  
# reading the file
data = my_file.read()
  
# replacing end splitting the text 
# when newline ('\n') is seen.
vocab = data.split("\n")
print(vocab)
my_file.close()

['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '##u', '##g', '##i', '##n', '##o', '##e', '##c', '##t', '##h', '##a', '##r', '##l', '##s', '##v', '##k', '##b', '##d', '##m', '##f', '##p', '##w', '##y', '##z', '##q', '##x', '##j', 'th', 'the', '##er', '##nd', '##in', '##ed', '##ou', '##at', '##en', 'and', '##es', '##or', '##on', 'of', 'to', '##is', '##ing', '##ar', '##it', '##an', '##as', '##ll', 'in', '##re', 'wh', '##om', 'he', 'be', 'ha', '##le', '##ot', '##ic', '##ut', '##ow', 'it', '##ld', 'that', '##ly', 'was', 'sh', '##gh', '##se', '##id', 'on', '##ve', '##ent', '##et', '##im', '##st', '##ion', '##ce', '##ir', '##ith', 'for', 'you', '##al', 'as', '##ay', 'his', '##ur', 'with', '##ter', 'st', '##ch', '##ver', 're', '##ad', 'her', 'an', '##ght', 'is', 'not', 'had', '##am', 'at', '##ct', '##her', '##oo', 'but', '##ess', '##ould', 'fr', 'se', 'pr', 'she', '

In [6]:
vocab = vocab[5:-1]
print(len(vocab))
vocab = vocab+[' '] # Add apostrophe later
print(len(vocab))
print(vocab)

999
1000
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '##u', '##g', '##i', '##n', '##o', '##e', '##c', '##t', '##h', '##a', '##r', '##l', '##s', '##v', '##k', '##b', '##d', '##m', '##f', '##p', '##w', '##y', '##z', '##q', '##x', '##j', 'th', 'the', '##er', '##nd', '##in', '##ed', '##ou', '##at', '##en', 'and', '##es', '##or', '##on', 'of', 'to', '##is', '##ing', '##ar', '##it', '##an', '##as', '##ll', 'in', '##re', 'wh', '##om', 'he', 'be', 'ha', '##le', '##ot', '##ic', '##ut', '##ow', 'it', '##ld', 'that', '##ly', 'was', 'sh', '##gh', '##se', '##id', 'on', '##ve', '##ent', '##et', '##im', '##st', '##ion', '##ce', '##ir', '##ith', 'for', 'you', '##al', 'as', '##ay', 'his', '##ur', 'with', '##ter', 'st', '##ch', '##ver', 're', '##ad', 'her', 'an', '##ght', 'is', 'not', 'had', '##am', 'at', '##ct', '##her', '##oo', 'but', '##ess', '##ould', 'fr', 'se', 'pr', 'she', 'we', 'sa', 'su', 'so', '##ill', '##ai

char_set = set()
for i in tqdm(range(len(metadata))):
    text = metadata.iloc[i,1]
    for char in text:
        char_set.add(char)

In [7]:
vocab = sorted(vocab)
num_embeddings = len(vocab)
print(num_embeddings)
vocab_ids = [int(i) for i in range(num_embeddings)]
vocab_dict = dict(zip(vocab,vocab_ids))
print(f"Character mappings : {vocab_dict}")

1000
Character mappings : {' ': 0, '##a': 1, '##ab': 2, '##able': 3, '##ac': 4, '##ace': 5, '##ach': 6, '##ack': 7, '##act': 8, '##ad': 9, '##ade': 10, '##ady': 11, '##ag': 12, '##age': 13, '##ail': 14, '##ain': 15, '##ained': 16, '##air': 17, '##ak': 18, '##ake': 19, '##aken': 20, '##aking': 21, '##al': 22, '##alk': 23, '##all': 24, '##ally': 25, '##als': 26, '##am': 27, '##ame': 28, '##amp': 29, '##an': 30, '##ance': 31, '##and': 32, '##ang': 33, '##ange': 34, '##ank': 35, '##ans': 36, '##ant': 37, '##ants': 38, '##ap': 39, '##aps': 40, '##ar': 41, '##ard': 42, '##are': 43, '##ared': 44, '##ark': 45, '##ars': 46, '##art': 47, '##ary': 48, '##as': 49, '##ase': 50, '##ash': 51, '##ason': 52, '##ass': 53, '##ast': 54, '##aster': 55, '##at': 56, '##ate': 57, '##ated': 58, '##ately': 59, '##ater': 60, '##ates': 61, '##ath': 62, '##ather': 63, '##ating': 64, '##ation': 65, '##ations': 66, '##atter': 67, '##au': 68, '##augh': 69, '##ause': 70, '##aut': 71, '##av': 72, '##ave': 73, '##ay': 7

In [8]:
feat_dict = {
        "sample_rate":16000,
        "n_mels":80, #as per reference
    }

# time_masks = [torchaudio.transforms.TimeMasking(time_mask_param=15, p=0.05) for _ in range(10)]
# train_transform = nn.Sequential(
#    torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=80, hop_length=160), #80 filter banks, 25ms window size, 10ms hop
#    torchaudio.transforms.FrequencyMasking(freq_mask_param=27),
#    *time_masks,
#  )
train_transform = torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=80, hop_length=160)
validation_transform = torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=80, hop_length=160)

In [9]:
def encode_word(word):
    tokens = []
    while len(word) > 0:
        i = len(word)
        while i > 0 and word[:i] not in vocab:
            i -= 1
        if i == 0:
            return ["[UNK]"]
        tokens.append(word[:i])
        word = word[i:]
        if len(word) > 0:
            word = f"##{word}"
    return tokens

In [10]:
def word_piece_tokenizer(text):
    text_list = re.split(" ", text)
    new_list = []
    for i in range(0,len(text_list)-1):
        new_list.append(text_list[i])
        new_list.append(' ')
    new_list.append(text_list[-1])   
    #print(new_list)
    encoded_words = []
    for word in new_list:
        encoded_words += encode_word(word)
    #print(encoded_words)
    mapped_words = [vocab_dict[word_piece] for word_piece in encoded_words]
    return mapped_words

In [11]:
class MyDataset(Dataset):
    """
    The Class will act as the container for our dataset. It will take your dataframe, the root path, and also the transform function for transforming the dataset.
    """
    def __init__(self, data_frame, char_dict, transform=None):
        self.data_frame = data_frame
        #self.root_dir = root_dir
        self.transform = transform
        #self.max_transcript_len = N
        self.char_dict = char_dict
    def __len__(self):
        # Return the length of the dataset
        return len(self.data_frame)
    
    def __getitem__(self, idx):
        # Return the observation based on an index. Ex. dataset[0] will return the first element from the dataset, in this case the image and the label.
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        file_path = self.data_frame.iloc[idx, 0]
        waveform, sample_rate = torchaudio.load(file_path, normalize = True)
        if sample_rate!=16000:
            waveform = transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)
        mel_spec = self.transform(waveform) # (batch, n_mels, time)
        mel_spec = mel_spec.squeeze(0).transpose(0,1)  # (time, n_mels)
        mel_spec_len = ((mel_spec.shape[0] - 1) // 2 - 1) // 2
        
        transcript = self.data_frame.iloc[idx,1]
        transcript = str(transcript)
        input_transcript_list = word_piece_tokenizer(transcript)
#         for char in transcript:
#             input_transcript_list.append(self.char_dict[char])
        
        label_tensor = torch.LongTensor(input_transcript_list)
        label_len = len(input_transcript_list)
    
        return mel_spec, label_tensor, mel_spec_len, label_len, transcript

In [12]:
# train_size = int(len(metadata)*0.99)
# dev_size = len(metadata) - train_size

# if not os.path.exists(metadata_path):
#     metadata_train = metadata[:train_size]
#     metadata_dev   = metadata[train_size:]
#     metadata_train.to_csv("lj_metadata_train.tsv", sep='\t', header=False, index=False)
#     metadata_dev.to_csv("lj_metadata_test.tsv", sep='\t', header=False, index=False)
# else:
#     print("Using existing metadata!")
#     metadata_train = pd.read_csv("lj_metadata_train.tsv", sep='\t', header=None, encoding='utf-8')
#     metadata_dev  = pd.read_csv("lj_metadata_test.tsv", sep='\t', header=None, encoding='utf-8')
    
trainset = MyDataset(metadata_train, vocab_dict, transform=train_transform)
devset = MyDataset(metadata_dev, vocab_dict, transform=validation_transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size = TRAIN_BS, shuffle = True, collate_fn = collate_batch, drop_last=True, num_workers=NUM_WORKERS, pin_memory=True)
devloader  = torch.utils.data.DataLoader(devset,  batch_size = TEST_BS, shuffle = True, collate_fn = collate_batch, drop_last=True, num_workers=NUM_WORKERS, pin_memory=True)

In [13]:
encoder_params = {
    "d_input": 80,
    "d_model": 144,
    "num_layers": 16,
    "conv_kernel_size": 32,
    "dropout": 0.1,
    "num_heads": 4
}

decoder_params = {
    "d_encoder": 144,
    "d_decoder": 320,
    "num_layers": 1,
    "num_classes":len(vocab)+1
}

In [14]:
encoder = ConformerEncoder(
                      d_input=encoder_params['d_input'],
                      d_model=encoder_params['d_model'],
                      num_layers=encoder_params['num_layers'],
                      conv_kernel_size=encoder_params['conv_kernel_size'], 
                      dropout=encoder_params['dropout'],
                      num_heads=encoder_params['num_heads']
                    )
  
decoder = LSTMDecoder(
                  d_encoder=decoder_params['d_encoder'], 
                  d_decoder=decoder_params['d_decoder'], 
                  num_layers=decoder_params['num_layers'],
                  num_classes= decoder_params['num_classes'])

In [15]:
encoder = encoder.to(DEVICE)
decoder = decoder.to(DEVICE)

In [16]:
char_decoder =  GreedyCharacterDecoder().eval()
criterion = nn.CTCLoss(blank=len(vocab), zero_infinity=True)
optimizer = torch.optim.AdamW(list(encoder.parameters()) + list(decoder.parameters()), lr=5e-4, betas=(.9, .98), eps = 1e-09, weight_decay=1e-6)
scheduler = TransformerLrScheduler(optimizer, encoder_params['d_model'], 10000)

In [17]:
model_size(encoder, 'Encoder')
model_size(decoder, 'Decoder')

Encoder - num_params: 10.13M,  size: 38.66MB
Decoder - num_params: 0.92M,  size: 3.5MB


In [18]:
gc.collect()

0

In [19]:
torch.cuda.set_device(DEVICE)
criterion = criterion.to(DEVICE)
char_decoder = char_decoder.to(DEVICE)
torch.cuda.empty_cache()

checkpoint_load_path = "conf_s_libri_best_wer.pt"

if(os.path.exists(checkpoint_load_path)):
    start_epoch, best_loss, best_wer = load_checkpoint(encoder, decoder, optimizer, scheduler, checkpoint_load_path, DEVICE)
    print(f'Resuming training from checkpoint starting at epoch {start_epoch}.')
    print(f'Current model WER : {best_wer}%')

In [20]:
def train(encoder, decoder, char_decoder, optimizer, scheduler, criterion, grad_scaler, train_loader, device): 
    wer = WordErrorRate()
    cer = CharErrorRate()
    
    encoder.train()
    decoder.train()
    avg_loss = 0
    avg_wer = 0
    avg_cer = 0
    batch_count = 0
    for batch in tqdm(train_loader):
        batch_count += 1
        scheduler.step()
        gc.collect()
        spectrograms, labels, input_lengths, label_lengths, references, mask = batch 

        spectrograms = spectrograms.squeeze(1).to(device)
        labels = labels.to(device)
        input_lengths = torch.tensor(input_lengths).to(device)
        label_lengths = torch.tensor(label_lengths).to(device)
        mask = mask.to(device)
    
        outputs, _ = encoder(spectrograms, mask)
        outputs, _ = decoder(outputs)
        loss = criterion(F.log_softmax(outputs, dim=-1).transpose(0, 1), labels, input_lengths, label_lengths)
        #loss.backward()
        
        grad_scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(encoder.parameters(), 0.1)
        torch.nn.utils.clip_grad_norm_(decoder.parameters(), 0.1)
        #if (i+1) % args.accumulate_iters == 0:
        grad_scaler.step(optimizer)
        grad_scaler.update()
        optimizer.zero_grad()
        #avg_loss.update(loss.detach().item())
        
        #use for small datasets
        
#         optimizer.step()
#         optimizer.zero_grad()
        
        avg_loss += loss.detach().item()
        
        inds = char_decoder(outputs.detach())
        # print("train shape:",inds.shape)
        predictions = []
        for sample in inds:
            #print(sample.shape)
            predictions.append(int_to_text(sample, len(vocab), vocab))
        avg_wer += wer(predictions, references) * 100
        avg_cer += cer(predictions, references) * 100

    avg_loss = avg_loss/batch_count
    avg_wer = avg_wer/batch_count
    avg_cer = avg_cer/batch_count
    print(f'Avg WER: {avg_wer}%, Avg Loss: {avg_loss}')  
    for i in range(5):
        print('Prediction: ', predictions[i])
        print('Reference: ', references[i])
    
    # Print metrics and predictions 
    del spectrograms, labels, input_lengths, label_lengths, references, outputs, inds, predictions
    return avg_wer, avg_cer, avg_loss
    
def validate(encoder, decoder, char_decoder, criterion, test_loader, device):
    ''' Evaluate model on test dataset. '''

    wer = WordErrorRate()
    cer = CharErrorRate()
        
    avg_loss = 0
    avg_wer = 0
    batch_count = 0
    avg_cer = 0
    
    encoder.eval()
    decoder.eval()
    for batch in tqdm(test_loader):
        gc.collect()
        batch_count += 1
        spectrograms, labels, input_lengths, label_lengths, references, mask = batch 
  
    # Move to GPU
        spectrograms = spectrograms.to(device)
        labels = labels.to(device)
        input_lengths = torch.tensor(input_lengths).to(device)
        label_lengths = torch.tensor(label_lengths).to(device)
        mask = mask.to(device)

        with torch.no_grad():
            outputs, _ = encoder(spectrograms, mask)
            outputs, _ = decoder(outputs)
            loss = criterion(F.log_softmax(outputs, dim=-1).transpose(0, 1), labels, input_lengths, label_lengths)
            avg_loss += loss.item()

            inds = char_decoder(outputs.detach())
            # print("validation shape:", inds.shape)
            predictions = []
            for sample in inds:
                predictions.append(int_to_text(sample, len(vocab), vocab))

            avg_wer += wer(predictions, references) * 100
            avg_cer += cer(predictions, references) * 100
    print(".............................TEST PREDICTIONS...........................")
    for i in range(5):
        print('Prediction: ', predictions[i])
        print('Reference: ', references[i])
    print("************************************************************************")
    
    return avg_wer/batch_count, avg_cer/batch_count, loss/batch_count 

In [21]:
checkpoint_save_path = "conf_s_libri_chk.pt"
best_loss_save_path = "conf_s_libri_best_loss.pt"
best_wer_save_path = "conf_s_libri_best_wer.pt"

In [22]:
best_loss = float('inf')
best_wer = float('inf')
optimizer.zero_grad()

use_amp = True
variational_noise_std = 0.0001

# Mixed Precision Setup
if use_amp:
    print('Using Mixed Precision')
grad_scaler = GradScaler(enabled=use_amp)

optimizer.zero_grad()
for epoch in range(start_epoch, EPOCHS):
    print(f"Epoch : {epoch+1}/{EPOCHS}")
    torch.cuda.empty_cache()
    
    #variational noise for regularization - COMMENTING ON MAY 19
#     add_model_noise(encoder, std=variational_noise_std, gpu=True)
#     add_model_noise(decoder, std=variational_noise_std, gpu=True)

    
    # Train/Validation loops
    #wer, cer, loss = train(encoder_parallel, decoder_parallel, char_decoder, optimizer, scheduler, criterion, trainloader, DEVICE) 
    wer, cer, loss = train(encoder, decoder, char_decoder, optimizer, scheduler, criterion, grad_scaler, trainloader, DEVICE) 
    valid_wer, valid_cer, valid_loss = validate(encoder, decoder, char_decoder, criterion, devloader, DEVICE)
    print(f'Epoch {epoch} - Valid WER: {valid_wer}%, Valid CER: {valid_cer}%, Valid Loss: {valid_loss}, Train WER: {wer}%, Train CER: {cer}%, Train Loss: {loss}')  
    
    # Save best model
    if valid_loss <= best_loss:
        print('Validation loss improved, saving best model.')
        best_loss = valid_loss
        save_checkpoint(encoder, decoder, optimizer, scheduler, valid_loss, epoch+1, best_loss_save_path, valid_wer, valid_cer, vocab_dict)
    
    if epoch%3==0:
        print(f'Saving checkpoint at epoch:{epoch+1}')
        save_checkpoint(encoder, decoder, optimizer, scheduler, valid_loss, epoch+1, checkpoint_save_path, valid_wer, valid_cer, vocab_dict)
    
    if valid_wer <= best_wer:
        print(f"Validation WER improved, saving best model.")
        best_wer = valid_wer
        save_checkpoint(encoder, decoder, optimizer, scheduler, valid_loss, epoch+1, best_wer_save_path, valid_wer, valid_cer, vocab_dict)

Using Mixed Precision
Epoch : 1/1000


100%|█████████████████████████████████████| 6224/6224 [1:06:06<00:00,  1.57it/s]


Avg WER: 98.96968078613281%, Avg Loss: 4.337714385059315
Prediction:  and r c r r r r r r r r c r c c r r r r
Reference:  aunt mauds appreciation of that to night was indeed managerial and the performers own contribution fairly that of the faultless soldier on parade densher saw himself for the moment as in his purchased stall at the play
Prediction:  and r r r r r c r r r r r to r r r r c
Reference:  then he scraped its claws with a penknife sharpened its nails fitted it with spurs of sharp steel spat on its head spat on its neck anointed it with spittle as they used to rub oil over athletes then set it down in the pit a redoubtable champion exclaiming
Prediction:  and r r r r r c r c r r r r r r r r r c
Reference:  while tom rawle quietly chuckling at the fat lads misfortune aided him to his feet down flat said mackinson again as he discerned several shadows moving across a space a considerable distance to the north of them
Prediction:  and r r r r r r r r r r r r r r r r r
Reference

100%|█████████████████████████████████████████| 121/121 [00:35<00:00,  3.45it/s]


.............................TEST PREDICTIONS...........................
Prediction:  and r 
Reference:  i am not allowed to perform magic except for my own amusement he told his visitors as he lighted a pipe with a crooked stem and began to smoke
Prediction:  and c
Reference:  one thinks one hears hydras talking
Prediction:  and the
Reference:  anders face grew red
Prediction:  and r
Reference:  he returned carrying his jumping shoes which are provided as you are aware with several sharp spikes
Prediction:  and r 
Reference:  i address him in italian and he answers very wittily but his way of speaking makes me smile and i tell him why
************************************************************************
Epoch 0 - Valid WER: 97.4828872680664%, Valid CER: 94.58130645751953%, Valid Loss: 0.042391058057546616, Train WER: 98.96968078613281%, Train CER: 88.01534271240234%, Train Loss: 4.337714385059315
Validation loss improved, saving best model.
Saving checkpoint at epoch:1
Validation W

...

Avg WER: 98.22488403320312%, Avg Loss: 3.8466739589803325
Prediction:  and r r r r de r r r r r r r r r d d
Reference:  when at last he got out of the house he saw with disgust the confounded policeman still standing on the pavement can he be waiting for me thought mister vladimir looking up and down for some signs of a hansom
Prediction:  and r r r a r r r r r r r d r r r d
Reference:  what if i do forbid it she walked a little forward leaving the carnation bed and halted under the shade of the dark cedar tree her heart and colour alike fading mister yorke followed and stood before her william i must do my duty
Prediction:  and  r r r r r r r r  r r r d d d
Reference:  every one has his own manner of growth and we have ours answered the young beeches this is the way its done where we come from and we are perhaps as good as you are
Prediction:  and r r r de r r r r r r r r r r d r
Reference:  broke out of him of a sudden and he flung up his arms like a lost spirit the monk took the hel

100%|█████████████████████████████████████████| 121/121 [00:34<00:00,  3.54it/s]


.............................TEST PREDICTIONS...........................
Prediction:  and r
Reference:  one of us always remains on board while the other is on shore
Prediction:  and r
Reference:  the proof was in three long slips i had left them all together
Prediction:  and r r
Reference:  asked phronsie in intense interest slipping down out of pollys arms and crowding up close to jaspers side
Prediction:  and          
Reference:  his housekeeper had the management of everything she never allowed him to be in need of anything and she gave no account of his money which she kept altogether because he never asked her to render any accounts
Prediction:  and r     
Reference:  by quick marches through these inaccessible mountains that general freed himself from the superior forces of the covenanters
************************************************************************
Epoch 1 - Valid WER: 97.35359954833984%, Valid CER: 92.20494079589844%, Valid Loss: 0.040320757776498795, Train WER: 9

  6%|██▎                                     | 369/6224 [03:16<51:50,  1.88it/s]


KeyboardInterrupt: 